In [1]:
import pandas as pd
import numpy as np

## Read data

In [95]:
data = pd.read_csv('mushrooms.csv')

In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

## Preprocessing

### Selecting only few columns

In [97]:
data = data[['cap-color', 'stalk-shape', 'population', 'class']]

### for cap-color ; selection only if cap-color = brown or red
- If brown --> 1
- else --> 0

In [98]:
data = data[(data['cap-color'] == 'n') | (data['cap-color'] == 'e' )]

In [99]:
data['cap-color'] = np.where(data['cap-color'] == 'n', 1, 0)

### For population
- If solitary --> 1
- else --> 0

In [100]:
data['population'] = np.where(data['population']=='y',1,0)

### For stalk-shape column
- if enlarge --> 1
- else --> 0

In [101]:
data['stalk-shape'] = np.where(data['stalk-shape']=='e', 1, 0)

### For class prediction

In [102]:
data['class'] = np.where(data['class']=='e', 1, 0)

#### Distribution

In [103]:
data['cap-color'].value_counts()

1    2284
0    1500
Name: cap-color, dtype: int64

In [104]:

data['stalk-shape'].value_counts()

0    3136
1     648
Name: stalk-shape, dtype: int64

In [105]:
data['population'].value_counts()

0    3116
1     668
Name: population, dtype: int64

In [106]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data[['cap-color', 'stalk-shape', 'population','class']].values, train_size=0.8, random_state=42)

In [107]:
train.shape,test.shape

((3027, 4), (757, 4))

## Decision tree 

### Gini Index calculation

In [108]:
def  GI(y):
    """
    y - Numpy array for which Gini impurity need to be calculated
    
    """
    # No more splits
    if (len(y) == 0):
        return 0
    
    p1 = np.sum(y==1)/len(y) # edible
    p0 = 1 - p1 # Not edible
    P = p1**2 + p0**2

    GI = 1 - P
    
    return GI

### Entropy Calculation

In [238]:
def  entropy(y):
    """
    y - Numpy array for which entropy need to be calculated
    
    """
    # No more splits
    if (len(y) == 0):
        return 0
    
    p1 = np.sum(y==1)/len(y) # edible
    p0 = 1 - p1 # Not edible
    
    # If  there exist a pure split, then there exist only one subset (either yes or no). So one probability will be 0.
    # If proability is 0, log(0) is not defined. Hence for calculation purpose 0 is returned as the entropy
    if p1 == 0 or p0 == 0:
        return 0
    entropy = - p1 * np.log2(p1) - p0 * np.log2(p0)
    
    return entropy

### Information gain calculation

In [253]:
def information_gain(Data, feature, criteria, thresh=1):
    """
    Data: matrix of  Remaininf data to be splitted with 3 features and labels
    feature: index of features based on which split is done:  possible values 0,1,2
    criteria: Whether to use Gini-Index or Entropy
    """
    if len(Data) == 0:
        return None
        
    LeftNode_index  = np.where(Data[: , feature] >= thresh)[0] 
    RightNode_index = np.where(Data[: , feature] < thresh)[0]
    
    LeftNode = Data[LeftNode_index, -1] # Empty data frame if no examples on right node
    RightNode = Data[RightNode_index, -1] # Empty data frame if no examples on right node

    # Main node entropy
    RootNode = Data[:, -1]
    E_root = criteria(RootNode)
    
    # Calculate entropy for left and right nodes.
    # Below subsets may/may'nt contain both edible and non edible mushrooms
    E_L = criteria(LeftNode) # feature value = 1
    E_R = criteria(RightNode) # feature value = 0
    
    # Weights, i.e subset proportion for left and right nodes
    w_L = len(LeftNode)/len(Data)
    w_R = len(RightNode)/len(Data)
    
    # Information gain
    IG = E_root - (w_L * E_L) - (w_R * E_R)
    
    return IG, LeftNode_index, RightNode_index


In [254]:
IG, left_index, right_index = information_gain(train, 0, GI)
IG#, left_index, right_index

0.009390220543105898

In [255]:
IG, left_index, right_index = information_gain(train, 0, entropy)
IG#, left_index, right_index

0.013606074784502153

----

In [256]:
IG, left_index, right_index = information_gain(train, 1, GI)
IG#, left_index, right_index

0.025783460888718968

In [257]:
IG, left_index, right_index = information_gain(train, 1, entropy)
IG#, left_index, right_index

0.03853124397558094

---

In [258]:
IG, left_index, right_index = information_gain(train, 2, GI)
IG#, left_index, right_index

0.10877250015287399

In [259]:
IG, left_index, right_index = information_gain(train, 2, entropy)
IG#, left_index, right_index

0.2058977188556389

### To get best split node at any instant

In [260]:
def get_best_split(data, n, criteria):
    """
    data -> Elements in the current root node
    n -> Number of features in dataframe
    criteria -> Choice of using Gini index or Entropy: GI - Gini Index or Entropy
    
    returns list containing: 
    Best split's Information gain value
    Best split's left node index
    Best split's right node index
    Best split feature number
    
    """
    Max_IG = [0,[],[], None]
    for i in range(n):
        IG, leftNode_idx, rightNode_idx = information_gain(data, i, criteria) 
        if IG >= Max_IG[0]:
            Max_IG = [IG, leftNode_idx, rightNode_idx, i]
    return Max_IG

In [261]:

len(data.iloc[np.where(data.iloc[: , 1] == 3)[0], :])

0

In [262]:
print(f"Best feature to use as root node is: {get_best_split(train, 3, GI)[3]}")

Best feature to use as root node is: 2


### Building decision tree
- **Stopping criteria**
    1. When there is no impurity at all split.
    1. When reaching a maximum specified depth. 
    1. When improvement in purity score is below certain threshold.
    1. When Number of examples in a node is below a threshold.
- I will be using **1 & 2** 
    - Say depth = 2

In [263]:
def build_tree(data, name, max_depth, current_depth, criteria=GI):
    if max_depth == current_depth:
        return
    if len(data) == 0:
        return
    _, leftNode_idx, rightNode_idx, feature = get_best_split(data, 3, criteria)
    leftNodeData = data[leftNode_idx, :]
    rightNodeData = data[rightNode_idx, :]
    
    if len(rightNodeData) == 0 or len(leftNodeData) == 0:
        print(f"purity achieved: {data.shape}")
        return
    
    print(data.shape, leftNodeData.shape, rightNodeData.shape)
    
    tree.append([name, feature]) # Node name, feature selected at that node
    
    build_tree(leftNodeData, "left", max_depth, current_depth+1, criteria)
    build_tree(rightNodeData, "right", max_depth, current_depth+1, criteria)

In [264]:
tree = []
build_tree(train,"root", 3, 0, criteria=GI)

tree

(3027, 4) (529, 4) (2498, 4)
purity achieved: (529, 4)
(2498, 4) (437, 4) (2061, 4)
(437, 4) (390, 4) (47, 4)
(2061, 4) (1113, 4) (948, 4)


[['root', 2], ['right', 1], ['left', 0], ['right', 0]]